In [11]:
import csv
import spacy
import random
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from difflib import get_close_matches
import difflib 

# Load CSV file into a list of dictionaries
businesses = []
with open('FurnitureDealers.csv', mode='r', encoding='utf-8-sig') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        businesses.append(row)

# Define intents and their corresponding keywords
intents = {
    'greetings': ['hi', 'hello', 'hey', 'greetings', 'good morning', 'good afternoon', 'good evening','how are you', 'how are you doing'],
    'business_info': ['information', 'details', 'contact', 'location', 'category', 'phone', 'email','county','business','company','companies','businesses'],
    'goodbye': ['bye', 'goodbye', 'see you later', 'talk to you later', 'until next time'],
    'information': ['tell', 'know', 'information', 'about']
}

# Define responses for each intent
responses = {
    'greetings': ['Hello!', 'Hi there!', 'Hey!', 'Nice to meet you.',],
    'business_info': ['Here is some information about the business:', 'Here are the details of the business:', 'This is what I found about the business:', 'Sure, here you go:'],
    'goodbye': ['Goodbye!', 'See you later!', 'Until next time.']

}

# Define function to preprocess user input
def preprocess_input(text):
    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub('[^A-Za-z]', ' ', text.lower())
    # Tokenize text and remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in word_tokenize(text) if token not in stop_words]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Define function to get business information based on query
def get_business_info(query):
    # Preprocess query
    query = preprocess_input(query)
    # Find matching businesses
    matching_businesses = []
    for business in businesses:
        if query in business['Business Name'].lower() or query in business['Business Segment'].lower() or query in business['Locality'].lower():
            matching_businesses.append(business)
    # Select a random business from the matching businesses
    if matching_businesses:
        selected_business = random.choice(matching_businesses)
        # Format business information as a string
        business_info_str = f"Name: {selected_business['Business Name']}\nCategory: {selected_business['Business Segment']}\nLocation: {selected_business['Locality']}\nPhone: {selected_business['Main Phone']}\nEmail: {selected_business['Email']}"
        return business_info_str
    else:
        return "Sorry, I could not find any businesses matching your query."

greetings_responses = [
    "Hello! How can I help you?",
    "Hi there! What can I do for you today?",
    "Hey! How can I assist you?"
]

greetings_responses = ['Hello!', 'Hi there!', 'Hey!', 'Hi!']
business_info_responses = [
    "Here's some information about {name}: {info}",
    "Sure! {name} is {info}.",
    "{name} is located in {location}. Here's some more information: {info}.",
    "I have some information about {name} here: {info}.",
    "{name} can be found in {location}. Here's what I know: {info}."
]


def get_business_info_details(name):
    matching_businesses = [b for b in businesses if name.lower() in b['Business Name'].lower()]
    if matching_businesses:
        selected_business = random.choice(matching_businesses)
        info = f"Category: {selected_business['Business Segment']}\nLocation: {selected_business['Locality']}\nPhone: {selected_business['Main Phone']}\nEmail: {selected_business['Email']}"
        return info, selected_business['Business Name'], selected_business['Locality']
    else:
        return None, None, None


business_names =[]
business_info =[]



<function get_business_info at 0x000001F30F3DFDC0>


In [5]:
nlp = spacy.load("en_core_web_sm")

def preprocess_input(text):
    # Perform any necessary preprocessing on the input text
    return text.lower()

def extract_entities(text):
    # Extract named entities from the input text using spaCy
    doc = nlp(text)
    entities = {}
    for ent in doc.ents:
        entities[ent.label_] = ent.text
    return entities



In [23]:
def get_bot_response(user_input):
    # Preprocess user input
    preprocessed_input = preprocess_input(user_input)
    entities = extract_entities(preprocessed_input)
    # Determine intent
    intent = 'unknown'
    for key, value in intents.items():
        if any(keyword in preprocessed_input for keyword in value):
            intent = key
            break
    # Generate bot response based on intent
    if intent == 'greetings':
        response = random.choice(greetings_responses)
    elif intent == 'business_info':
        if 'name' in entities:
            name = entities['Business Name']
            if name in business_names:
                response = business_info[business_names.index(name)]
            else:
                closest_match = difflib.get_close_matches(name, business_names, n=1, cutoff=0.6)
                if closest_match:
                    response = business_info[business_names.index(closest_match[0])]
                else:
                    response = "Sorry, I couldn't find any business matching that name."
        elif 'Business Segment' in entities:
            category = entities['Business Segment']
            businesses = [business_info[i] for i in range(len(business_info)) if category in business_info[i]['Business Segment']]
            if businesses:
                response = random.choice(businesses)
            else:
                response = f"Sorry, I couldn't find any businesses in the {category} category."
        else:
            response = "What information are you looking for?"
    elif "Tell me about" in preprocessed_input:
        query = preprocessed_input.replace("tell me about", "").strip()
        business_info_str = get_business_info(preprocessed_input)
        if business_info_str:
           response = random.choice(responses['business_info']) + '\n' + business_info_str
        else:
            response = "I'm sorry, I couldn't find any business matching that name."
    else:
        response = "I'm sorry, I didn't understand your question. Can you please rephrase it?"
    
    return response


# Example usage
user_input = "Hello"
bot_response = get_bot_response(user_input)
print(bot_response)

Hey!
